In [1]:
import pandas as pd

In [7]:
import joblib
from functions import get_player_id, get_players, previsao_real
from model_presentation import build_feature_state


model = joblib.load("models/tennis_winner_model.joblib")
matches_list = []
for year in range(2020, 2025):
    m = pd.read_csv(f'./data/atp_matches_{year}.csv',low_memory=False)
    matches_list.append(m)
all_matches = pd.concat(matches_list, ignore_index=True)
all_matches['tourney_date'] = pd.to_datetime(all_matches['tourney_date'], format='%Y%m%d')
state = build_feature_state(all_matches)
players = get_players()

2025-11-05 19:47:49.185 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-05 19:47:49.186 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 19:47:49.285 
  command:

    streamlit run C:\Users\Pedro Mesquita\AppData\Roaming\Python\Python313\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-05 19:47:49.286 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 19:47:49.287 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 19:47:49.288 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 19:47:49.451 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 19:47:49.4

In [120]:
from dataclasses import dataclass
from typing import List, Tuple, Union

# ---------- MODELO DE DADOS ----------
@dataclass
class Match:
    p1: str
    p2: str
    winner: str | None = None

class Bracket:
    def __init__(self, pairs: List[Tuple[str, str]]):
        if len(pairs) == 0 or (len(pairs) & (len(pairs)-1)) != 0:
            raise ValueError("Número de jogos da rodada deve ser potência de 2 (ex.: 64, 32, 16...).")
        self.rounds: List[List[Match]] = [[Match(a, b) for a, b in pairs]]

    def current_round(self) -> List[Match]:
        return self.rounds[-1]

    def advance(self, winners: List[Union[str, int, bool]]) -> List[Tuple[str, str]]:
        """
        winners pode ser:
          - lista de nomes dos vencedores (mesmo texto do par);
          - ou lista de 0/1 (0 = p1 vence; 1 = p2 vence);
          - ou lista de bool (False=p1, True=p2).
        Retorna a lista de pares da próxima rodada.
        """
        cur = self.current_round()
        if len(winners) != len(cur):
            raise ValueError("Tamanho de 'winners' diferente do número de jogos da rodada.")

        next_names: List[str] = []
        for match, w in zip(cur, winners):
            if isinstance(w, str):
                if w == match.p1:
                    match.winner = match.p1
                elif w == match.p2:
                    match.winner = match.p2
                else:
                    raise ValueError(f"Vencedor '{w}' não encontrado no jogo ({match.p1} vs {match.p2}).")
            elif isinstance(w, bool) or isinstance(w, int):
                idx = int(w)
                if idx not in (0, 1):
                    raise ValueError("Valores inteiros em winners devem ser 0 (p1) ou 1 (p2).")
                match.winner = match.p1 if idx == 0 else match.p2
            else:
                raise TypeError("Cada item de 'winners' deve ser str, int (0/1) ou bool.")
            next_names.append(match.winner)

        # monta próxima rodada: (1 vs 2), (3 vs 4), ...
        nxt_pairs = [(next_names[i], next_names[i+1]) for i in range(0, len(next_names), 2)]
        self.rounds.append([Match(a, b) for a, b in nxt_pairs])
        return nxt_pairs

    def round_pairs(self, round_index: int = -1) -> List[Tuple[str, str]]:
        """Retorna pares (p1, p2) da rodada indicada (default: rodada atual)."""
        return [(m.p1, m.p2) for m in self.rounds[round_index]]

    def pretty_print(self, round_index: int = -1) -> None:
        round_index -= 1
        r = self.rounds[round_index]
        print(f"Rodada {round_index+1} ({len(r)} jogos)")
        for i, m in enumerate(r, 1):
            tag = f"  [{i:02d}] {m.p1}  vs  {m.p2}"
            if m.winner:
                tag += f"   -> vencedor: {m.winner}"
            print(tag)

    def print_full_bracket(self) -> None:
        """
        Exibe todas as rodadas do torneio (até 7) em um formato de bracket textual.
        """
        max_rounds = len(self.rounds)
        # padding horizontal por rodada
        pad = 25
        # altura máxima
        total_matches = len(self.rounds[0])
        lines = [[" " * pad * max_rounds for _ in range(total_matches * 2)]]

        # Constrói uma matriz textual simples
        matrix = [[" " * (pad * max_rounds) for _ in range(total_matches * 2)] for _ in range(2)]

        for r_i, round_ in enumerate(self.rounds):
            col = r_i * pad
            step = 2 ** (r_i + 1)
            half = step // 2
            for m_i, match in enumerate(round_):
                base_row = m_i * step
                name1 = f"{match.p1[:22]:<22}"
                name2 = f"{match.p2[:22]:<22}"
                w = match.winner or ""
                lines_needed = max_rounds * 2
                print(f"{' ' * col}{name1}")
                print(f"{' ' * col}{name2}")
                if w:
                    print(f"{' ' * (col + 2)}→ {w}")

            print()

        print("=" * (pad * max_rounds))
        print("🏆 TORNEIO COMPLETO")
        print("=" * (pad * max_rounds))
        for i, round_ in enumerate(self.rounds, 1):
            print(f"Rodada {i}: {len(round_)} jogos")



In [136]:
FIRST_ROUND: List[Tuple[str, str]] = [
    ("Jannik Sinner", "Nicolas Jarry"),
    ("Tristan Schoolkate", "Taro Daniel"),
    ("Marcos Giron", "Yannick Hanfmann"),
    ("Tomas Martin Etcheverry", "Flavio Cobolli"),
    ("Hubert Hurkacz", "Tallon Griekspoor"),
    ("Miomir Kecmanovic", "Dusan Lajovic"),
    ("Matteo Berrettini", "Cameron Norrie"),
    ("Holger Rune", "Zhizhen Zhang"),
    ("Alex Michelsen", "Stefanos Tsitsipas"),
    ("James Mccabe", "Martin Landaluce"),
    ("Gabriel Diallo", "Luca Nardi"),
    ("Karen Khachanov", "Adrian Mannarino"),
    ("Francisco Cerundolo", "Alexander Bublik"),
    ("Facundo Diaz Acosta", "Zizou Bergs"),
    ("Tristan Boyer", "Federico Coria"),
    ("Alex De Minaur", "Botic Van De Zandschulp"),
    ("Taylor Fritz", "Jenson Brooksby"),
    ("Cristian Garin", "Borna Coric"),
    ("Daniel Altmaier", "Francisco Comesana"),
    ("Gael Monfils", "Giovanni Mpetshi Perricard"),
    ("Ben Shelton", "Brandon Nakashima"),
    ("Pablo Carreno Busta", "Kamil Majchrzak"),
    ("Denis Shapovalov", "Roberto Bautista Agut"),
    ("Lorenzo Musetti", "Matteo Arnaldi"),
    ("Joao Fonseca", "Andrey Rublev"),
    ("Lorenzo Sonego", "Stan Wawrinka"),
    ("Fabian Marozsan", "Thiago Seyboth Wild"),
    ("Frances Tiafoe", "Arthur Rinderknech"),
    ("Corentin Moutet", "Alexei Popyrin"),
    ("Mitchell Krueger", "Rinky Hijikata"),
    ("Learner Tien", "Camilo Ugo Carabelli"),
    ("Daniil Medvedev", "Kasidit Samrej"),
    ("Novak Djokovic", "Nishesh Basavareddy"),
    ("Jaime Faria", "Pavel Kotov"),
    ("Reilly Opelka", "Gauthier Onclin"),
    ("Tomas Machac", "Sumit Nagal"),
    ("Jiri Lehecka", "Li Tu"),
    ("Hugo Gaston", "Omar Jasika"),
    ("Benjamin Bonzi", "David Goffin"),
    ("Francesco Passaro", "Grigor Dimitrov"),
    ("Jack Draper", "Mariano Navone"),
    ("Thanasi Kokkinakis", "Roman Safiullin"),
    ("Aleksandar Vukic", "Damir Dzumhur"),
    ("Sebastian Korda", "Lukas Klein"),
    ("Jordan Thompson", "Dominik Koepfer"),
    ("Nuno Borges", "Alexandre Muller"),
    ("Yoshihito Nishioka", "Aziz Dougaz"),
    ("Carlos Alcaraz", "Alexander Shevchenko"),
    ("Casper Ruud", "Jaume Munar"),
    ("Jakub Mensik", "Nikoloz Basilashvili"),
    ("Alejandro Davidovich Fokina", "Juncheng Shang"),
    ("Felix Auger Aliassime", "Jan Lennard Struff"),
    ("Roberto Carballes Baena", "Alejandro Tabilo"),
    ("James Duckworth", "Dominic Stricker"),
    ("Kei Nishikori", "Thiago Monteiro"),
    ("Tommy Paul", "Christopher Oconnell"),
    ("Ugo Humbert", "Matteo Gigante"),
    ("Hady Habib", "Bu Yunchaokete"),
    ("Quentin Halys", "Adam Walton"),
    ("Arthur Fils", "Otto Virtanen"),
    ("Arthur Cazaux", "Sebastian Baez"),
    ("Jacob Fearnley", "Nick Kyrgios"),
    ("Pedro Martinez", "Luciano Darderi"),
    ("Alexander Zverev", "Lucas Pouille"),
]

bracket = Bracket(FIRST_ROUND)


In [137]:
for round in range(7):
    winners = []
    print(f'Simulando round {round+1}')
    nxt = FIRST_ROUND if round == 0 else nxt
    for matchup in nxt:
        proba,probb = previsao_real(get_player_id(matchup[0], players), get_player_id(matchup[1], players), 'Rápida', 3, 32, model, state)
        winners.append(matchup[0] if proba>probb else matchup[1])
    try:
        nxt = bracket.advance(winners)
    except:
        print(f'Vencedor do torneio: {winners[0]}\nFinal jogada entre {matchup[0]} e {matchup[1]}')

Simulando round 1
Simulando round 2
Simulando round 3
Simulando round 4
Simulando round 5
Simulando round 6
Simulando round 7
Vencedor do torneio: Jannik Sinner
Final jogada entre Jannik Sinner e Carlos Alcaraz
